In [42]:
using Pkg
Pkg.activate("..")

# Load module
push!(LOAD_PATH, "../src")
using WriterVerifier

using Random
Random.seed!(42);

  Activating project at `c:\Users\igorp\WriterVerifier`


In [43]:
# Load data
println("Loading images...")

image_folder = "../data/words"

# Load images
writers = load_images(image_folder; max_per_writer=200);

Loading images...
Loaded 76 writers
f01: 200 images
d01: 200 images
g05: 200 images
e07: 200 images
e02: 200 images
a04: 200 images
p02: 200 images
p06: 200 images
r02: 200 images
h07: 200 images
b04: 200 images
k02: 200 images
d03: 145 images
d06: 200 images
c03: 200 images
b01: 200 images
c04: 200 images
f03: 200 images
c02: 200 images
g06: 200 images
l07: 200 images
j07: 200 images
k04: 200 images
e06: 200 images
l03: 200 images
a03: 200 images
r03: 200 images
n06: 200 images
j04: 200 images
r06: 200 images
n02: 200 images
g03: 200 images
k01: 86 images
j06: 200 images
h04: 200 images
m02: 200 images
b05: 200 images
g02: 200 images
k03: 200 images
c01: 200 images
f02: 200 images
p03: 200 images
p01: 200 images
a05: 200 images
d05: 200 images
m01: 200 images
g04: 200 images
h06: 200 images
f04: 200 images
n01: 200 images
j01: 200 images
b02: 200 images
b03: 200 images
l04: 200 images
b06: 200 images
n04: 200 images
e01: 200 images
l01: 200 images
a02: 200 images
a01: 200 images
h02: 

In [ ]:
# Create pairs
pairs, labels = create_pairs(writers; positive=200, negative=200);

Creating positive pairs...
Creating negative pairs...
Created 298 pairs
Positive: 149
Negative: 149


In [45]:
# Create model
model = create_model()
println("Model created")


Model created


In [ ]:
# Model training

trained_model, history = train_model!(
    model, pairs, labels;
    epochs=5, 
    batch_size=8,
    learning_rate=0.00001
)

final_acc = round(history["val_acc"][end] * 100, digits=1)
println("Final accuracy: $(final_acc)%")

Starting training...
Training data: 238
Validation data: 60

Epoch 1/5
Train loss: 0.7094
Val loss: 0.695
Val accuracy: 50.0%

Epoch 2/5
Train loss: 0.7101
Val loss: 0.6951
Val accuracy: 50.0%

Epoch 3/5
Train loss: 0.7051
Val loss: 0.6949
Val accuracy: 50.0%

Epoch 4/5
Train loss: 0.6946
Val loss: 0.6949
Val accuracy: 50.0%

Epoch 5/5
Train loss: 0.7076
Val loss: 0.6949
Val accuracy: 50.0%

Training completed!
Final accuracy: 50.0%


In [47]:
# Testing on real examples

# Select random test examples
idx = randperm(length(pairs))[1:min(6, length(pairs))]
test_pairs = pairs[idx]
test_labels = labels[idx]

for i in 1:length(test_pairs)
    path1, path2 = test_pairs[i]
    true_label = test_labels[i]
    
    # Extract writer names
    writer1 = split(basename(path1), "-")[1]
    writer2 = split(basename(path2), "-")[1]
    
    # Test similarity
    similarity = test_similarity(trained_model, path1, path2)
    
    # Interpret results
    prediction = similarity > 0.5 ? "SAME" : "DIFFERENT"
    truth = true_label == 1 ? "SAME" : "DIFFERENT"
    correct = (similarity > 0.5) == (true_label == 1) ? "✓" : "X"
    
    println("$writer1 and $writer2")
    println("Similarity: $(round(similarity, digits=3))")
    println("Prediction: $prediction | Truth: $truth $correct")
end


a01 and a01
Similarity: 0.506
Prediction: SAME | Truth: SAME ✓
k04 and b02
Similarity: 0.513
Prediction: SAME | Truth: DIFFERENT X
d04 and b06
Similarity: 0.513
Prediction: SAME | Truth: DIFFERENT X
e04 and e04
Similarity: 0.511
Prediction: SAME | Truth: SAME ✓
g04 and c01
Similarity: 0.524
Prediction: SAME | Truth: DIFFERENT X
b02 and b02
Similarity: 0.518
Prediction: SAME | Truth: SAME ✓


In [48]:
# Model saving
try
    if !isdir("../models")
        mkdir("../models")
    end
    
    save_model(trained_model, "../models/model.jld2")
    
catch e
    println("Save error: $e")
end

Model saved: ../models/model.jld2
